# Exploring Russian Lexicon-based Tokenizer with no spaces


In [1]:
import os, sys
cwd = os.getcwd()
project_path = cwd[:cwd.find('pygents')+7]
if project_path not in sys.path: sys.path.append(project_path)
os.chdir(project_path) 

#from importlib import reload  # Python 3.4+

import pickle
import pandas as pd

#force reimport
if 'pygents.util' in sys.modules:
    del sys.modules['pygents.util']
if 'pygents.text' in sys.modules:
    del sys.modules['pygents.text']
if 'pygents.plot' in sys.modules:
    del sys.modules['pygents.plot']
if 'pygents.token' in sys.modules:
    del sys.modules['pygents.token']
if 'pygents.token_plot' in sys.modules:
    del sys.modules['pygents.token_plot']


from pygents.token import *
from pygents.text import *
from pygents.util import *
from pygents.plot import plot_bars, plot_dict, matrix_plot
from pygents.token_plot import *

In [2]:
path = 'data/corpora/Russian/'
test_df = pd.read_csv(os.path.join(path,'magicdata/zh_en_ru_100/CORPUS_ZH_EN_RU.txt'),delimiter='\t')
test_texts = list(test_df['ru'])
print(len(test_texts))
test_df[['ru']]

100


,ru
0,Как насчет медицинской страховки? В случае ваш...
1,"Для тех, у кого есть страховка, по договору ст..."
2,Необходимо осознать важность наличия страховки.
3,"На самом деле, это явление действительно очень..."
4,Этому поколению родителей действительно необхо...
...,...
95,Ресо предлагает не только страхование автомоби...
96,"Однако при покупке дома, за исключением первон..."
97,Этот вид финансовых вложений имеет определенны...
98,"Если ваша инвестиционная ориентация верна, то ..."


In [3]:
for text in test_texts:
    print(text)

Как насчет медицинской страховки? В случае вашей семьи, её можно оформить и взрослому и ребенку.
Для тех, у кого есть страховка, по договору страхования они получат компенсацию в размере 300 тысяч рублей.
Необходимо осознать важность наличия страховки.
На самом деле, это явление действительно очень распространено, например, для страхования от несчастных случаев, чем больше вы покупаете, тем больше страхуете.
Этому поколению родителей действительно необходимо покупать страховку.
Ну, прямо сейчас действительно желательно купить страховку.
Машину нужно покупать в полном объеме, а дом можно купить в кредит.
Вы можете купить страховку, страховка, конечно же, делится на множество категорий.
Медицинская страховка очень важна.
Эту часть денег платит страховая компания.
Послушайте, я не знаю, слышали ли вы когда-нибудь об этом, это страховка в Сбере.
Покупка дома на самом деле является инвестицией.
Вы когда-нибудь узнавали об обучении в Альфе?
Если он депонирован в банке, каков результат сложны

In [4]:
del_tokenizer = DelimiterTokenizer()


In [5]:
#get raw lexicon list
ru_lex = list(pd.read_csv("https://raw.githubusercontent.com/aigents/aigents-java/master/lexicon_russian.txt",sep='\t',header=None).to_records(index=False))
print(len(ru_lex))

#debug raw lexicon
print(max(ru_lex,key=lambda item:item[1]))
ru_lex_dict = weightedlist2dict(ru_lex,lower=False) # no case-insensitive merge
print(len(ru_lex_dict))
print(ru_lex_dict['не'])
print(ru_lex_dict['Не'])
print(ru_lex_dict['НЕ'])

# merge and get top weight
ru_lex_dict = weightedlist2dict(ru_lex,lower=True) # with case-insensitive merge
print(len(ru_lex_dict))
print(ru_lex_dict['не'])
top_weight = max([ru_lex_dict[key] for key in ru_lex_dict],key=lambda item:item)
print(top_weight)

# add delimiters to the list
ru_lex_delimited = ru_lex + [(i, top_weight) for i in list(delimiters)]
print(len(delimiters))
print(len(ru_lex_delimited)) 


100000
('не', 3164900)
100000
3164900
218183
2075
91396
3385158
3385158
34
100034


In [6]:
# no delimiters
filter_thresholds = [0,0.00001,0.0001,0.001,0.01]
for t in filter_thresholds:
    lex = listofpairs_compress_with_loss(ru_lex,t) if t > 0 else ru_lex
    ru_lex0_tokenizer = LexiconIndexedTokenizer(lexicon=lex,sortmode=0,cased=True)
    ru_lex1_tokenizer = LexiconIndexedTokenizer(lexicon=lex,sortmode=1,cased=True)
    ru_lex2_tokenizer = LexiconIndexedTokenizer(lexicon=lex,sortmode=2,cased=True)
    print(t,ru_lex0_tokenizer.count_params())
    print(evaluate_tokenizer_f1(test_texts,del_tokenizer,ru_lex0_tokenizer,nospaces=True,debug=False))#sort by len
    print(evaluate_tokenizer_f1(test_texts,del_tokenizer,ru_lex1_tokenizer,nospaces=True,debug=False))#sort by freq
    print(evaluate_tokenizer_f1(test_texts,del_tokenizer,ru_lex2_tokenizer,nospaces=True,debug=False))#sort by len and freq
    print()


0 91396
0.7
0.24
0.66

1e-05 91396
0.7
0.24
0.66

0.0001 46290
0.65
0.26
0.64

0.001 5391
0.45
0.26
0.44

0.01 399
0.29
0.23
0.29



In [7]:
# with delimiters
filter_thresholds = [0,0.00001,0.0001,0.001,0.01]
for t in filter_thresholds:
    lex = listofpairs_compress_with_loss(ru_lex_delimited,t) if t > 0 else ru_lex_delimited
    ru_lex0_tokenizer = LexiconIndexedTokenizer(lexicon=lex,sortmode=0,cased=True)
    ru_lex1_tokenizer = LexiconIndexedTokenizer(lexicon=lex,sortmode=1,cased=True)
    ru_lex2_tokenizer = LexiconIndexedTokenizer(lexicon=lex,sortmode=2,cased=True)
    print(t,ru_lex0_tokenizer.count_params())
    print(evaluate_tokenizer_f1(test_texts,del_tokenizer,ru_lex0_tokenizer,nospaces=True,debug=False))#sort by len
    print(evaluate_tokenizer_f1(test_texts,del_tokenizer,ru_lex1_tokenizer,nospaces=True,debug=False))#sort by freq
    print(evaluate_tokenizer_f1(test_texts,del_tokenizer,ru_lex2_tokenizer,nospaces=True,debug=False))#sort by len and freq
    print()


0 91430
0.72
0.29
0.68

1e-05 91430
0.72
0.29
0.68

0.0001 43863
0.7
0.31
0.68

0.001 4994
0.5
0.33
0.5

0.01 408
0.37
0.32
0.37



In [8]:
ru_lex0_tokenizer = LexiconIndexedTokenizer(lexicon=ru_lex_delimited,sortmode=0,cased=True)
for text in test_texts:
    expected = del_tokenizer.tokenize(text)
    remove_all(expected,' ')
    actual = ru_lex0_tokenizer.tokenize(text.replace(' ',''))
    f1 = calc_f1(expected,actual)
    if f1 < 1:
        print(expected)
        print(actual)
        print(round(f1,2))


['Как', 'насчет', 'медицинской', 'страховки', '?', 'В', 'случае', 'вашей', 'семьи', ',', 'её', 'можно', 'оформить', 'и', 'взрослому', 'и', 'ребенку', '.']
['Как', 'насчет', 'медицинской', 'страховки', '?', 'Вс', 'луча', 'ева', 'ш', 'ей', 'семьи', ',', 'её', 'можно', 'оформить', 'ив', 'з', 'росло', 'му', 'ире', 'бен', 'ку', '.']
0.54
['Для', 'тех', ',', 'у', 'кого', 'есть', 'страховка', ',', 'по', 'договору', 'страхования', 'они', 'получат', 'компенсацию', 'в', 'размере', '300', 'тысяч', 'рублей', '.']
['Для', 'тех', ',', 'ук', 'ого', 'есть', 'страховка', ',', 'подо', 'говору', 'страхования', 'они', 'получат', 'компенсацию', 'враз', 'мере', '300', 'тысяч', 'рублей', '.']
0.7
['На', 'самом', 'деле', ',', 'это', 'явление', 'действительно', 'очень', 'распространено', ',', 'например', ',', 'для', 'страхования', 'от', 'несчастных', 'случаев', ',', 'чем', 'больше', 'вы', 'покупаете', ',', 'тем', 'больше', 'страхуете', '.']
['Наса', 'мо', 'м', 'деле', ',', 'это', 'явление', 'действительно', 'о

['Небольшая', 'проблема', 'не', 'равносильна', 'отсутствию', 'проблем', ',', 'потому', 'что', 'ваша', 'кредитная', 'линия', 'не', 'зависит', 'от', 'того', ',', 'сколько', 'у', 'вас', 'кредитных', 'карт', '.']
['Небольшая', 'проблема', 'неравно', 'сильна', 'отсутствию', 'проблем', ',', 'потому', 'что', 'ваша', 'кредит', 'на', 'я', 'линия', 'не', 'зависит', 'оттого', ',', 'сколько', 'ув', 'ас', 'кредитных', 'карт', '.']
0.68
['Например', ',', 'квалификационный', 'аттестат', 'брокера-дилера', ',', 'аттестат', 'банкира', '.']
['Например', ',', 'кв', 'али', 'фи', 'ка', 'ци', 'он', 'ный', 'аттестат', 'б', 'рок', 'ер', 'а', '-', 'дилера', ',', 'аттестат', 'банкира', '.']
0.48
['На', 'самом', 'деле', ',', 'я', 'думаю', ',', 'что', 'покупка', 'страховки', 'также', 'выгодна', '.', 'Вы', 'можете', 'расценивать', 'это', 'как', 'экономию', 'денег', '.']
['Наса', 'мо', 'м', 'деле', ',', 'яду', 'маю', ',', 'что', 'покупка', 'страховки', 'также', 'выгодна', '.', 'Вы', 'можете', 'расценивать', 'это', '

['Ресо', 'предлагает', 'не', 'только', 'страхование', 'автомобиля', ',', 'я', 'также', 'видел', 'множество', 'страховок', '.']
['Ре', 'со', 'предлагает', 'нет', 'о', 'ль', 'кострах', 'ова', 'ни', 'е', 'автомобиля', ',', 'я', 'также', 'видел', 'множество', 'страхов', 'ок', '.']
0.5
['Однако', 'при', 'покупке', 'дома', ',', 'за', 'исключением', 'первоначального', 'взноса', ',', 'ежемесячная', 'ипотека', 'приравнивается', 'к', 'арендной', 'плате', '.']
['Однако', 'при', 'покупке', 'дома', ',', 'за', 'исключением', 'первоначального', 'взноса', ',', 'еже', 'мес', 'яч', 'на', 'яи', 'потек', 'ап', 'р', 'ира', 'в', 'нива', 'ет', 'сяк', 'арендной', 'плате', '.']
0.6
['Если', 'ваша', 'инвестиционная', 'ориентация', 'верна', ',', 'то', 'она', 'имеет', 'гораздо', 'большее', 'преимущество', ',', 'чем', 'депозиты', '.']
['Если', 'ваша', 'инвестиционная', 'ориентация', 'верна', ',', 'тоо', 'на', 'имеет', 'гораздо', 'большее', 'преимущество', ',', 'чем', 'депо', 'зи', 'ты', '.']
0.76
